In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt

In [ ]:
sol = pd.read_csv('../out/solution-test2.csv')
sol

In [ ]:
cities_geom = gpd.read_file('../data/us-major-cities/USA_Major_Cities.shp')
cols = ['NAME', 'ST', 'geometry']
cities_geom[cols]


In [ ]:
sol_cols = sol.columns
sol_origin = sol.merge(cities_geom[cols], how='inner', left_on=['city_origin', 'state_origin'], right_on=['NAME', 'ST']).rename(columns={'geometry': 'pt_origin'})

sol_od = sol_origin.merge(cities_geom[cols], how='inner', left_on=['city_dest', 'state_dest'], right_on=['NAME', 'ST']).rename(columns={'geometry': 'pt_dest'})
sol_geom = sol_od[['pt_origin', 'pt_dest']]
sol_geom['geometry'] = sol_geom.apply(lambda row: shapely.LineString((row['pt_origin'], row['pt_dest'])), axis='columns')
sol_geom

In [ ]:
og_crs = cities_geom.crs
final = gpd.GeoDataFrame(sol, geometry=sol_geom['geometry'], crs=og_crs) # type: ignore
states = gpd.read_file('../data/us-states/States_shapefile.shp').to_crs(epsg=3395)
# remove AK and HI
states: gpd.GeoDataFrame = states[(states['State_Code'] != 'AK') & (states['State_Code'] != 'HI')] # type: ignore
citiesPlotMask = (cities_geom['POPULATION'] > 200e3) & (cities_geom['ST'] != 'HI') & (cities_geom['ST'] != 'AK')
cities_for_plot = cities_geom[citiesPlotMask].to_crs(epsg=3395)

fig, ax1 = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True, figsize=(20, 20))
ax1 = states.plot(ax=ax1, cmap='Pastel2')
ax1 = cities_for_plot.plot(ax=ax1, marker='o', color='black')
final.to_crs(epsg=3395).plot(ax=ax1, cmap='rainbow')

In [ ]:
final['construction_cost_usd'].sum() / 1e9